<a href="https://colab.research.google.com/github/uday-routhu/week5/blob/master/Project_2_Part_1_Core_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 2 - Part 1 (Core):

* Author: Udayakumar Routhu

###First choice: dataset 1


1. Source of data

2. Brief description of data

3. What is the target?

4. What does one row represent? (A person? A business? An event? A product?)

5. Is this a classification or regression problem?

6. How many features does the data have?

7. How many rows are in the dataset?

8. What, if any, challenges do you foresee in cleaning, exploring, or modeling this dataset?


In [ ]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',100)

import missingno as msno
## Setting the max_columns to 50
pd.set_option('display.max_columns',50)

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Classification Metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, \
f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression

from sklearn import set_config
set_config(transform_output='pandas')

# Set global scikit-learn configuration
from sklearn import set_config
# Display estimators as a diagram
set_config(display='diagram') # '

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

###Define EDA Functions

###explore_categorical


In [ ]:
# def explore_categorical_check_constant(df, x, fillna = True, placeholder = 'MISSING',
def explore_categorical(df, x, fillna = True, placeholder = 'MISSING',
                        figsize = (6,4), order = None):
  """BEST VERSION"""
  # Make a copy of the dataframe and fillna
  temp_df = df.copy()


  ## Save null value counts and percent for printing
  null_count = temp_df[x].isna().sum()
  null_perc = null_count/len(temp_df)* 100


  # fillna with placeholder
  if fillna == True:
    temp_df[x] = temp_df[x].fillna(placeholder)


  # Create figure with desired figsize
  fig, ax = plt.subplots(figsize=figsize)

  ## Plotting a count plot
  sns.countplot(data=temp_df, x=x, ax=ax, order=order)

  # Rotate Tick Labels for long names
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

  # Add. atitle with the feature name included
  ax.set_title(f"Column: {x}", fontweight='bold')

  # Fix layout and show plot (before print statements)
  fig.tight_layout()
  plt.show()


  # Print null value info
  print(f"- NaN's Found: {null_count} ({round(null_perc,2)}%)")
  # Print cardinality info
  nunique = temp_df[x].nunique()
  print(f"- Unique Values: {nunique}")


  # Get the most most common value, its count as # and as %
  most_common_val_count = temp_df[x].value_counts(dropna=False).head(1)
  most_common_val = most_common_val_count.index[0]
  freq = most_common_val_count.values[0]

  perc_most_common = freq / len(temp_df) * 100
  print(f"- Most common value: '{most_common_val}' occurs {freq} times ({round(perc_most_common,2)}%)")

  # print message if quasi-constant or constant (most common val more than 98% of data)
  if perc_most_common > 98:
    print(f"\n- [!] Warning: '{x}' is a constant or quasi-constant feature and should be dropped.")

  return fig, ax

In [ ]:
def plot_categorical_vs_target(df, x, y,figsize=(6,4),
                            fillna = True, placeholder = 'MISSING',
                            order = None):
  # Make a copy of the dataframe and fillna
  temp_df = df.copy()
  # # Save null values before imputing
  # null_count = temp_df[x].isna().sum()
  # null_perc = null_count/len(df)* 100

  ## fillna with placeholder
  if fillna == True:
    temp_df[x] = temp_df[x].fillna(placeholder)

  # or drop nulls prevent unwanted 'nan' group in stripplot
  else:
    temp_df = temp_df.dropna(subset=[x])


  ## Create the figure and subplots
  fig, ax = plt.subplots(figsize=figsize)

    # Barplot
  sns.barplot(data=temp_df, x=x, y=y, ax=ax, order=order, alpha=0.6,
              linewidth=1, edgecolor='black', errorbar=None)

  # Boxplot
  sns.stripplot(data=temp_df, x=x, y=y, hue=x, ax=ax,
                order=order, hue_order=order, legend=False,
                edgecolor='white', linewidth=0.5,
                size=3,zorder=0)

  # Rotate xlabels
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

  # Add a title
  ax.set_title(f"{x} vs. {y}", fontweight='bold')
  fig.tight_layout()
    # show fig and print
  plt.show()
  # print(f"- NaN's Found: {null_count} ({round(null_perc,2)}%)")

  return fig, ax

###explore_numeric

In [ ]:
# TO DO: add the new print statements from explore_categorical
def explore_numeric(df, x, figsize=(6,5) ):

  ## Save null value counts and percent for printing
  null_count = df[x].isna().sum()
  null_perc = null_count/len(df)* 100


  ## Making our figure with gridspec for subplots
  gridspec = {'height_ratios':[0.7,0.3]}
  fig, axes = plt.subplots(nrows=2, figsize=figsize,
                           sharex=True, gridspec_kw=gridspec)
  # Histogram on Top
  sns.histplot(data=df, x=x, ax=axes[0])

  # Boxplot on Bottom
  sns.boxplot(data=df, x=x, ax=axes[1])

  ## Adding a title
  axes[0].set_title(f"Column: {x}", fontweight='bold')

  ## Adjusting subplots to best fill Figure
  fig.tight_layout()

  # Ensure plot is shown before message
  plt.show()


  # Print null value info
  print(f"- NaN's Found: {null_count} ({round(null_perc,2)}%)")
  # Print cardinality info
  nunique = df[x].nunique()
  print(f"- Unique Values: {nunique}")


  # Get the most most common value, its count as # and as %
  most_common_val_count = df[x].value_counts(dropna=False).head(1)

  most_common_val = most_common_val_count.index[0]
  freq = most_common_val_count.values[0]
  perc_most_common = freq / len(df) * 100

  print(f"- Most common value: '{most_common_val}' occurs {freq} times ({round(perc_most_common,2)}%)")

  # print message if quasi-constant or constant (most common val more than 98% of data)
  if perc_most_common > 98:
    print(f"\n- [!] Warning: '{x}' is a constant or quasi-constant feature and should be dropped.")

  return fig, axes

In [ ]:
def plot_numeric_vs_target(df, x, y,figsize=(6,4),):
  # Calculate the correlation
  corr = df[[x,y]].corr().round(2)
  r = corr.loc[x,y]

  # Plot the data
  fig, ax = plt.subplots(figsize=figsize)
  scatter_kws={'ec':'white','lw':1,'alpha':0.8}
  sns.regplot(data=df, x=x, y=y, ax=ax, scatter_kws=scatter_kws)

  ## Add the title with the correlation
  ax.set_title(f"{x} vs. {y} (r = {r})", fontweight='bold')

  ## Print message with info on the count and % of null values
  null_count = df[x].isna().sum()
  if null_count > 0:
    null_perc = null_count/len(df)* 100
    print(f"- NaN's Found: {null_count} ({round(null_perc,2)}%)")

  return fig, ax

### Load in the dataset

In [ ]:
# Load the data set
fpath_adult = "/content/drive/MyDrive/CodingDojo/02-MachineLearning/Week08/Data/adult.csv"
df_adult = pd.read_csv(fpath_adult)

In [ ]:
df_adult.info()
df_adult.head()

### Source of data
  The data can be downloaded from [this link](https://www.kaggle.com/datasets/wenruliu/adult-income-dataset)

###Brief description of data
- An individual’s annual income results from various factors. Intuitively, it is influenced by the individual’s education level, age, gender, occupation, and etc.

###What is the target?
- Income

###What does one row represent? (A person? A business? An event? A product?)
- A person

###Is this a classification or regression problem?
 - Since the goal is to predict whether an individual's income is above or below a threshold (categorical outcome), this is a classification problem.

###How many features does the data have?  
 - Number of attributes: 14
 - These are the demographics and other features to describe a person

###How many rows are in the dataset?
- Number of rows: 48842

###What, if any, challenges do you foresee in cleaning, exploring, or modeling this dataset?
 - we clean and explore the data before running the models

###CLean and Prepare The Data

###Identified and addressed missing values.

In [ ]:
df_adult.isna().sum()

* There are no null values to impute place holder

###Deleted duplicate rows.

In [ ]:
#Explore Data
df_adult.duplicated().sum()

* There are 52 duplicated records in dataset

In [ ]:
# drop duplicated record
df_adult = df_adult.drop_duplicates()

In [ ]:
#again check duplicates dropped or not
df_adult.duplicated().sum()

* now, there are no duplicated rows

###Check for impossible numeric values/Delete unnecessary columns.

In [ ]:
df_adult.info()

* there is no data type missing

###Identified and corrected inconsistencies in data for categorical values (i.e. Cat, cat, cats).

In [ ]:
data_types = df_adult.dtypes
str_cols = data_types[data_types=='object'].index
for col in str_cols:
    print(f'- {col}:')
    print(df_adult[col].value_counts(dropna=False))
    print("\n\n")
    print(df_adult[col])

* There is specail char '?' availble in 'relationship','workclass' column so removing that value

In [ ]:
values_to_remove = ['?']
# Remove rows containing specified values in the 'year' column
df_adult = df_adult[~df_adult['relationship'].isin(values_to_remove)]
df_adult = df_adult[~df_adult['workclass'].isin(values_to_remove)]

In [ ]:
data_types = df_adult.dtypes
str_cols = data_types[data_types=='object'].index
for col in str_cols:
    print(f'- {col}:')
    print(df_adult[col].value_counts(dropna=False))
    print("\n\n")
    print(df_adult[col])

* Now , there are no inconsistency columns

###Check for and address any impossible values in numeric columns.

In [ ]:
stats =  df_adult.describe()
stats.loc[['mean','min','max']]

#####Outliers

* There are no impossible numeric values
* no extreme outliers are noted

In [ ]:
df_adult.describe(include='object')

###Explore the data

#### Define EDA Functions

#####workclass

In [ ]:
# Confirming the dtype to select correct EDA function
feature = "workclass"
df_adult[feature].dtype

In [ ]:
df_adult.info()

In [ ]:
# Testing function
fig, ax = explore_categorical(df_adult, "workclass", figsize=(5,3))

Observations

1. What type of feature is it? (Nominal category, ordinal, numeric)
  - Categorical (nominal)

2. How many null values? What percentage? What would you do with the null values (drop the rows? drop the column? impute? if impute, with what?)
  - 0 null values.
  - No need to impute.

3. Is the feature constant or quasi-constant?
  - No, the most common category makes up only 73.62% of the feature.

4. What is the cardinality? Is it high?
  - There are 8 categories. Not high cardinality ( < 10 categories)

5. Would we know this BEFORE the target is determined?
  - Yes.

6. Is there a business case/understanding reason to exclude based on our business case?
  - No.


####education

In [ ]:
# Testing function
fig, ax = explore_categorical(df_adult, "education", figsize=(5,3))

Observations

1. What type of feature is it? (Nominal category, ordinal, numeric)
  - Categorical (nominal)

2. How many null values? What percentage? What would you do with the null values (drop the rows? drop the column? impute? if impute, with what?)
  - 0 null values.
  - No need to impute.

3. Is the feature constant or quasi-constant?
  - No, the most common category makes up only 32.53% of the feature.

4. What is the cardinality? Is it high?
  - There are 16 categories. high cardinality ( > 10 categories)

5. Would we know this BEFORE the target is determined?
  - Yes.

6. Is there a business case/understanding reason to exclude based on our business case?
  - No.

####marital-status

In [ ]:
# Testing function
fig, ax = explore_categorical(df_adult, "marital-status", figsize=(5,3))

Observations

1. What type of feature is it? (Nominal category, ordinal, numeric)
  - Categorical (nominal)

2. How many null values? What percentage? What would you do with the null values (drop the rows? drop the column? impute? if impute, with what?)
  - 0 null values.
  - No need to impute.

3. Is the feature constant or quasi-constant?
  - No, the most common category makes up only 46.61% of the feature.

4. What is the cardinality? Is it high?
  - There are 7 categories.No high cardinality ( < 10 categories)

5. Would we know this BEFORE the target is determined?
  - Yes.

6. Is there a business case/understanding reason to exclude based on our business case?
  - No.

####occupation

In [ ]:
# Testing function
fig, ax = explore_categorical(df_adult, "occupation", figsize=(5,3))

Observations

1. What type of feature is it? (Nominal category, ordinal, numeric)
  - Categorical (nominal)

2. How many null values? What percentage? What would you do with the null values (drop the rows? drop the column? impute? if impute, with what?)
  - 0 null values.
  - No need to impute.

3. Is the feature constant or quasi-constant?
  - No, the most common category makes up only 46.61% of the feature.

4. What is the cardinality? Is it high?
  - There are 15 categories.No high cardinality ( > 10 categories)

5. Would we know this BEFORE the target is determined?
  - Yes.

6. Is there a business case/understanding reason to exclude based on our business case?
  - No.

####relationship

In [ ]:
# Testing function
fig, ax = explore_categorical(df_adult, "relationship", figsize=(5,3))

Observations

1. What type of feature is it? (Nominal category, ordinal, numeric)
  - Categorical (nominal)

2. How many null values? What percentage? What would you do with the null values (drop the rows? drop the column? impute? if impute, with what?)
  - 0 null values.
  - No need to impute.

3. Is the feature constant or quasi-constant?
  - No, the most common category makes up only 41.29% of the feature.

4. What is the cardinality? Is it high?
  - There are 6 categories.No high cardinality ( < 10 categories)

5. Would we know this BEFORE the target is determined?
  - Yes.

6. Is there a business case/understanding reason to exclude based on our business case?
  - No.

####race

In [ ]:
# Testing function
fig, ax = explore_categorical(df_adult, "race", figsize=(5,3))

Observations

1. What type of feature is it? (Nominal category, ordinal, numeric)
  - Categorical (nominal)

2. How many null values? What percentage? What would you do with the null values (drop the rows? drop the column? impute? if impute, with what?)
  - 0 null values.
  - No need to impute.

3. Is the feature constant or quasi-constant?
  - No, the most common category makes up only 85.68% of the feature.

4. What is the cardinality? Is it high?
  - There are 5 categories.No high cardinality ( > 10 categories)

5. Would we know this BEFORE the target is determined?
  - Yes.

6. Is there a business case/understanding reason to exclude based on our business case?
  - No.

####gender

In [ ]:
# Testing function
fig, ax = explore_categorical(df_adult, "gender", figsize=(5,3))

Observations

1. What type of feature is it? (Nominal category, ordinal, numeric)
  - Categorical (nominal)

2. How many null values? What percentage? What would you do with the null values (drop the rows? drop the column? impute? if impute, with what?)
  - 0 null values.
  - No need to impute.

3. Is the feature constant or quasi-constant?
  - No, the most common category makes up only 67.59% of the feature.

4. What is the cardinality? Is it high?
  - There are 2 categories.No high cardinality ( > 10 categories)

5. Would we know this BEFORE the target is determined?
  - Yes.

6. Is there a business case/understanding reason to exclude based on our business case?
  - No.

###Numeric

####age

In [ ]:
# Testing the function
explore_numeric(df_adult, "age");

**Observations:**

1. What type of feature is it? (categorical (nominal), ordinal, numeric?)
  - Numeric.

2. How many null values? What percentage? What will you do with the null values (drop the rows? drop the column? impute? if impute, with what?)
  - 0 null values.
  - No need to impute.
3. Is the feature constant or quasi-constant?
  - No.

4. What is the cardinality? Is it high?
  - Numeric feature, so not a concern.

5. Would we know this BEFORE the target is determined?
  - Yes.

6. Is there a business case/understanding reason to exclude based on our business case?
  - No

In [ ]:
# Testing function
fig, ax = plot_categorical_vs_target(df_adult, 'age','income')

- Feature vs. Target Observations:
  - Based on your business understanding, would you expect this feature to be a predictor of the target?
    - Yes.
  - Does this feature appear to be a predictor of the target?
    - Yes, but there are some outleirs that may be affecting the overall trendline.

###fnlwgt

In [ ]:
# Testing the function
explore_numeric(df_adult, "fnlwgt");

**Observations:**

1. What type of feature is it? (categorical (nominal), ordinal, numeric?)
  - Numeric.

2. How many null values? What percentage? What will you do with the null values (drop the rows? drop the column? impute? if impute, with what?)
  - 0 null values.
  - No need to impute.
3. Is the feature constant or quasi-constant?
  - No.

4. What is the cardinality? Is it high?
  - Numeric feature, so not a concern.

5. Would we know this BEFORE the target is determined?
  - Yes.

6. Is there a business case/understanding reason to exclude based on our business case?
  - No

In [ ]:
# Testing function
fig, ax = plot_categorical_vs_target(df_adult, 'fnlwgt','income')

- Feature vs. Target Observations:
  - Based on your business understanding, would you expect this feature to be a predictor of the target?
    - Yes.
  - Does this feature appear to be a predictor of the target?
    - Yes, but there are some outleirs that may be affecting the overall trendline.

###educational-num

In [ ]:
# Testing the function
explore_numeric(df_adult, "educational-num");

**Observations:**

1. What type of feature is it? (categorical (nominal), ordinal, numeric?)
  - Numeric.

2. How many null values? What percentage? What will you do with the null values (drop the rows? drop the column? impute? if impute, with what?)
  - 0 null values.
  - No need to impute.
3. Is the feature constant or quasi-constant?
  - No.

4. What is the cardinality? Is it high?
  - Numeric feature, so not a concern.

5. Would we know this BEFORE the target is determined?
  - Yes.

6. Is there a business case/understanding reason to exclude based on our business case?
  - No

In [ ]:
# Testing function
fig, ax = plot_categorical_vs_target(df_adult, 'educational-num','income')

- Feature vs. Target Observations:
  - Based on your business understanding, would you expect this feature to be a predictor of the target?
    - Yes.
  - Does this feature appear to be a predictor of the target?
    - Yes, but there are some outleirs that may be affecting the overall trendline.

###capital-gain

In [ ]:
# Testing the function
explore_numeric(df_adult, "capital-gain");

**Observations:**

1. What type of feature is it? (categorical (nominal), ordinal, numeric?)
  - Numeric.

2. How many null values? What percentage? What will you do with the null values (drop the rows? drop the column? impute? if impute, with what?)
  - 0 null values.
  - No need to impute.
3. Is the feature constant or quasi-constant?
  - No.

4. What is the cardinality? Is it high?
  - Numeric feature, so not a concern.

5. Would we know this BEFORE the target is determined?
  - Yes.

6. Is there a business case/understanding reason to exclude based on our business case?
  - No

In [ ]:
# Testing function
fig, ax = plot_categorical_vs_target(df_adult, 'capital-gain','income')

- Feature vs. Target Observations:
  - Based on your business understanding, would you expect this feature to be a predictor of the target?
    - Yes.
  - Does this feature appear to be a predictor of the target?
    - Yes, but there are some outleirs that may be affecting the overall trendline.

###capital-loss

In [ ]:
# Testing the function
explore_numeric(df_adult, "capital-loss");

**Observations:**

1. What type of feature is it? (categorical (nominal), ordinal, numeric?)
  - Numeric.

2. How many null values? What percentage? What will you do with the null values (drop the rows? drop the column? impute? if impute, with what?)
  - 0 null values.
  - No need to impute.
3. Is the feature constant or quasi-constant?
  - No.

4. What is the cardinality? Is it high?
  - Numeric feature, so not a concern.

5. Would we know this BEFORE the target is determined?
  - Yes.

6. Is there a business case/understanding reason to exclude based on our business case?
  - No

In [ ]:
# Testing function
fig, ax = plot_categorical_vs_target(df_adult, 'capital-loss','income')

- Feature vs. Target Observations:
  - Based on your business understanding, would you expect this feature to be a predictor of the target?
    - Yes.
  - Does this feature appear to be a predictor of the target?
    - Yes, but there are some outleirs that may be affecting the overall trendline.

###hours-per-week

In [ ]:
# Testing the function
explore_numeric(df_adult, "hours-per-week");

**Observations:**

1. What type of feature is it? (categorical (nominal), ordinal, numeric?)
  - Numeric.

2. How many null values? What percentage? What will you do with the null values (drop the rows? drop the column? impute? if impute, with what?)
  - 0 null values.
  - No need to impute.
3. Is the feature constant or quasi-constant?
  - No.

4. What is the cardinality? Is it high?
  - Numeric feature, so not a concern.

5. Would we know this BEFORE the target is determined?
  - Yes.

6. Is there a business case/understanding reason to exclude based on our business case?
  - No

In [ ]:
# Testing function
fig, ax = plot_categorical_vs_target(df_adult, 'hours-per-week','income')

- Feature vs. Target Observations:
  - Based on your business understanding, would you expect this feature to be a predictor of the target?
    - Yes.
  - Does this feature appear to be a predictor of the target?
    - Yes, but there are some outleirs that may be affecting the overall trendline.

####Produced univariate visuals for the target and all features.

In [ ]:
#Plot the counts for the number of adults who has income vs not having income.
fig, ax = plt.subplots()

survived_counts = df_adult['income'].value_counts()
plt.bar(['non-income', 'income'], survived_counts)
#plt.bar(['Died', 'Survived'], survived_counts)

# Set x-label, y-label, and title
plt.xlabel('income Status')
plt.ylabel('Count')
plt.title('non-income vs income')

# Display the plot
plt.show()

In [ ]:
#Plot a histogram of the distribution of Age.
ax = df_adult['age'].hist(bins = 30, edgecolor = 'black')
ax.tick_params(axis='x', rotation = 45)
ax.ticklabel_format(style='plain')
ax.set_title('Distribution of Age')
ax.set_xlabel('Age')
ax.set_ylabel('Number age group')

In [ ]:
# Create the boxplot
ax = sns.boxplot(data=df_adult,x="hours-per-week")
ax.set_xlabel("group");
ax.set_ylabel("hours-per-week");
ax.set_title("Total working hours");

####choose two visualizations from your analysis

####histogram

In [ ]:
# Create the histogram
ax = df_adult['income'].hist(bins =20, edgecolor = 'black')
ax.set_title('Distribution of Income')
ax.set_xlabel('Income')
ax.set_ylabel('Number of Individuals');

- Individuals with significant capital gains could be a potential target for higher-income financial products.
- They might consider targeting individuals in the age range where the probability of earning >50K is higher, or focus on promoting higher education to improve the chances of higher income.

####boxplot

In [ ]:
# Create the boxplot
ax = sns.boxplot(x = 'hours-per-week', y = 'income', data = df_adult)
ax.set_title('INCOME by work-hours');

- Individuals earning >50K tend to work slightly more hours per week on average compared to those earning <=50K. The distribution of hours worked per week is more spread out for the >50K group.

####Plot a corrleation

In [ ]:
plt.figure(figsize=(28, 18))
ax = sns.countplot(data=df_adult, hue='income', x='occupation');
ax.set(title='# of income - by occupation');

- Individuals with higher educational-num tend to have a higher likelihood of earning >50K. The distribution of educational-num is shifted to the right for individuals earning >50K, indicating a higher educational level.

#### Multivariate Visual

#### 1) Plot the counts for the number of income by Sex

In [ ]:
ax = sns.countplot(data=df_adult, hue='gender', x='income');
ax.set(title='# of Income - by Gender');

​Question: Which group had a higher income ?
- Answer: Male has higher income.

### 2) Plot a barplot of the average

In [ ]:
## functionize a barplot and call it plot_bar

def plot_bar(data,x, y, figsize=(20,5)):
    ## Make a larger fig/ax before plotting
    fig, ax = plt.subplots(figsize=figsize)

    ## Plot barplot

    ## counts, counts up the categories, sort the values, then puts them in a
    ## data frame.

    counts = df_adult[x].value_counts().sort_values(ascending=False).to_frame()
    ## ploting the barplot

    sns.barplot(data=counts,x=counts.index, y = counts[x],ax=ax)

    plt.xticks(rotation= 90)
    ax.set_title(f'Categorical Frequencies of {y}', fontsize = 20, fontweight = 'bold');
    ax.set_xlabel(f'Categories for {y}', fontsize = 15, fontweight = 'bold')
    ax.set_ylabel('Counts', fontsize = 15, fontweight = 'bold')

    return fig,ax

In [ ]:
fig, ax = plot_bar(df_adult, 'income', 'Experience income');
print(df_adult['income'].value_counts())

- Majority of people got <=50k income  

In [ ]:
ax = sns.barplot(data=df_adult,x='income', y='hours-per-week')
ax.set_title("Earnings based on hours of wrok");


- people who worked more than 40 hours, got higher salary  

### 3) Plot a corrleation heatmap of all of the numeric columns.

In [ ]:
# Mapping dictionary
income_mapping = {'<=50K': 0, '>50K': 1}

# Create a new DataFrame with the transformed 'income' column
df_transformed = df_adult.copy()  # Make a copy of the original DataFrame
df_transformed['income'] = df_adult['income'].map(income_mapping)

In [ ]:
corr = df_transformed.corr(numeric_only=True)
ax = sns.heatmap(corr,annot=True, cmap='coolwarm');
ax.set(title='Correlation Heatmap')

Question: which column had the highest positive correlation to the "Income" column?
- Answer: the income column was most positively correlated with the hours-per-week column.

###Modeling

### **Check Class Balance**

In [ ]:
df_adult['income'].value_counts(normalize = False)

In [ ]:
# Check how many samples of each class are present
df_adult['income'].value_counts(normalize=True)

### **Split the Data**

In [ ]:
# Define X and y
target = 'income'
X = df_adult.drop(columns = [target]).copy()
y = df_adult[target]

In [ ]:
# train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


#### Class Balance

In [ ]:
# Check how many samples of each class are present for train
y_train.value_counts(normalize=True)

### **Transformers**

In [ ]:
# Categorical Preprocessing Pipeline
# Save list of column names
ohe_cols = X_train.select_dtypes('object').columns
print("OneHotEncoder Columns:", ohe_cols)
# Instantiate the individual preprocessors
impute_na = SimpleImputer(strategy='constant', fill_value = "Missing")
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# Make pipeline with imputer and encoder
ohe_pipe = make_pipeline(impute_na, ohe_encoder)
# Making a ohe_tuple for ColumnTransformer
ohe_tuple = ('categorical', ohe_pipe, ohe_cols)
# Numerical Preprocessing Pipeline
# Save list of column names
num_cols = X_train.select_dtypes("number").columns
print("Numeric Columns:", num_cols)
# instantiate preprocessors
impute_median = SimpleImputer(strategy='median')
scaler = StandardScaler()
# Make a numeric preprocessing pipeline
num_pipe = make_pipeline(impute_median, scaler)
# Making a numeric tuple for ColumnTransformer
num_tuple = ('numeric', num_pipe, num_cols)
# Create the Column Transformer
preprocessor = ColumnTransformer([num_tuple, ohe_tuple],
                                 verbose_feature_names_out=False)

#### Evaluation Functions

In [ ]:
def classification_metrics(y_true, y_pred, label='',
                           output_dict=False, figsize=(8,4),
                           normalize='true', cmap='Blues',
                           colorbar=False):
  # Get the classification report
  report = classification_report(y_true, y_pred)
  ## Print header and report
  header = "-"*70
  print(header, f" Classification Metrics: {label}", header, sep='\n')
  print(report)
  ## CONFUSION MATRICES SUBPLOTS
  fig, axes = plt.subplots(ncols=2, figsize=figsize)
  # create a confusion matrix  of raw counts
  ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                normalize=None, cmap='gist_gray', colorbar=colorbar,
                ax = axes[0],);
  axes[0].set_title("Raw Counts")
  # create a confusion matrix with the test data
  ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                normalize=normalize, cmap=cmap, colorbar=colorbar,
                ax = axes[1]);
  axes[1].set_title("Normalized Confusion Matrix")
  # Adjust layout and show figure
  fig.tight_layout()
  plt.show()
  # Return dictionary of classification_report
  if output_dict==True:
    report_dict = classification_report(y_true, y_pred, output_dict=True)
    return report_dict

def evaluate_classification(model, X_train, y_train, X_test, y_test,
                            figsize=(6,4), normalize='true', output_dict = False,
                            cmap_train='Blues', cmap_test="Reds",colorbar=False):
  # Get predictions for training data
  y_train_pred = model.predict(X_train)
  # Call the helper function to obtain regression metrics for training data
  results_train = classification_metrics(y_train, y_train_pred, #verbose = verbose,
                                         output_dict=True, figsize=figsize,
                                         colorbar=colorbar, cmap=cmap_train,
                                         label='Training Data')
  print()
  # Get predictions for test data
  y_test_pred = model.predict(X_test)
  # Call the helper function to obtain regression metrics for test data
  results_test = classification_metrics(y_test, y_test_pred, #verbose = verbose,
                                        output_dict=True,figsize=figsize,
                                        colorbar=colorbar, cmap=cmap_test,
                                        label='Test Data' )
  if output_dict == True:
    # Store results in a dataframe if ouput_frame is True
    results_dict = {'train':results_train,
                    'test': results_test}
    return results_dict

####Model - 1

- DecisionTreeClassifier

In [ ]:
# Create an instance of the model
dtc = DecisionTreeClassifier(random_state=42)
# Create a model pipeline
clf_pipe = make_pipeline(preprocessor, dtc)
# Fit the model
clf_pipe.fit(X_train, y_train)

In [ ]:
clf_pipe.predict(X_train)


In [ ]:
y_train

In [ ]:
## Evaluate the model
evaluate_classification(clf_pipe, X_train, y_train, X_test, y_test)

- The model appears to be performing very well on the training data with perfect precision, recall, and F1-score.
- However, on the test data, the performance is slightly lower, indicating that the model might be overfitting to the training data.

###Using GridSearch CV

In [ ]:
# Define the parameter grid to search
param_grid = {
    'decisiontreeclassifier__max_depth': [None, 10, 20],
    'decisiontreeclassifier__min_samples_split': [2, 5, 10],
    'decisiontreeclassifier__min_samples_leaf': [1, 2, 4]
}

In [ ]:
# Create GridSearchCV instance
grid_search = GridSearchCV(clf_pipe, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_


In [ ]:
# Evalaute the best model
best_logreg = grid_search.best_estimator_
evaluate_classification(best_logreg, X_train, y_train, X_test, y_test)

- Applying GridSearchCV has led to improvements in the model's performance on both the training and test data. The model now achieves higher precision, recall, and F1-score for both classes.
- The accuracy has also improved, indicating that the model's generalization capability has improved as well.

#### Using PCA

In [ ]:
# Create a transformer pipeline
transformer_pca_dcn = make_pipeline(preprocessor, PCA(n_components=7))

In [ ]:
# Create a modeling pipeline
dtc_pipe_pca = make_pipeline(transformer_pca_dcn, dtc)
dtc_pipe_pca.fit(X_train, y_train)

In [ ]:
dtc_pipe_pca.predict(X_train)

In [ ]:
## Evaluate the model
evaluate_classification(dtc_pipe_pca, X_train, y_train, X_test, y_test)

- Using PCA with a DecisionTreeClassifier has resulted in a model with perfect performance on the training data, as indicated by precision, recall, and F1-score values of 1.00 for both classes.
- This suggests that the model has potentially overfit the training data, given the perfect scores.
- However, the model's performance on the test data is not as strong. While the model is still accurate, the precision, recall, and F1-score values for the '>50K' class are lower than those for the '<=50K' class.
- This indicates that the model's ability to predict the '>50K' class is not as reliable as its ability to predict the '<=50K' class

### Model -2

- Logistic Regression

In [ ]:
# Make an instance of the model with default parameters
logreg = LogisticRegression(max_iter=500, random_state=42)
# Put scaler and model in a pipeline
logreg_pipe = make_pipeline(preprocessor, logreg)
# Training the model on the data, storing the information learned from the data
# Model is learning the relationship between X and y
logreg_pipe.fit(X_train, y_train)

In [ ]:
logreg_pipe.predict(X_train)

In [ ]:
y_train

In [ ]:
## Evaluate the model
evaluate_classification(logreg_pipe, X_train, y_train, X_test, y_test)

- The Logistic Regression model seems to be performing consistently on both the training and test data.
- The precision, recall, and F1-score values for both classes are similar between the training and test sets, indicating that the model's performance is stable across different data subsets.

### Using GridSearch CV

In [ ]:
# Making a logreg pipe that uses L2 regularization
logreg_pipe = make_pipeline(preprocessor, LogisticRegression(solver='liblinear',
                                                       max_iter=500,
                                                       penalty='l2',
                                                       random_state=42))
# Define the params and instantiate gridsearch
l2_params = {'logisticregression__solver': ['lbfgs', 'liblinear', 'newton-cg',
                                            'newton-cholesky', 'sag', 'saga'],
                  'logisticregression__penalty' : ['l2'],
                  'logisticregression__C': [0.0001, 0.001, 0.01, 0.1,
                                            1, 10, 100, 1000,10000] }
gs = GridSearchCV(logreg_pipe, l2_params,verbose=True, n_jobs=-1,
                  return_train_score=True,scoring='recall_macro')
# Fit gridsearch and display best params
gs.fit(X_train, y_train)
gs.best_params_

Fitting 5 folds for each of 54 candidates, totalling 270 fits


In [ ]:
# Evalaute the best model
best_logreg = gs.best_estimator_
evaluate_classification(best_logreg, X_train, y_train, X_test, y_test)

- The precision, recall, and F1-score values are similar between the two datasets, indicating that the model is not overfitting.
- An accuracy of 0.85 suggests that the model's predictions are relatively accurate, though there's a trade-off between precision and recall, particularly for the '>50K' class.

####Create a PCA Pipeline with Regression

In [ ]:
# Create a transformer pipeline
transformer_pca = make_pipeline(preprocessor, PCA(n_components=7))

In [ ]:
# Create a modeling pipeline
logreg_pipe_pca = make_pipeline(transformer_pca, logreg)
logreg_pipe_pca.fit(X_train, y_train)

In [ ]:
logreg_pipe_pca.predict(X_train)

In [ ]:
## Evaluate the model
evaluate_classification(logreg_pipe_pca, X_train, y_train, X_test, y_test)

- The use of PCA along with Regression has resulted in a model with slightly lower accuracy compared to the previous models.
- The precision, recall, and F1-score values are consistent across training and test datasets, indicating that the model generalizes well to new data.
- The model seems to be better at predicting the '<=50K' class, as indicated by higher precision and recall values for that class.
-  However, the model's performance on the '>50K' class is relatively weaker, with lower precision and recall.

#### Choose your "production" model and justify this decision using the metrics most important to your business problem.

1. DecisionTreeClassifier:

   - Training Data Accuracy: 100%
   - Test Data Accuracy: 81%

2. GridSearchCV (with DecisionTreeClassifier):
   - Training Data Accuracy: 87%
   - Test Data Accuracy: 85%

3. PCA with DecisionTreeClassifier:
  - Training Data Accuracy: 100%
  - Test Data Accuracy: 78%

4. LogisticRegression:

  - Training Data Accuracy: 85%
  - Test Data Accuracy: 85%

5. PCA (with Regression):
  - Training Data Accuracy: 83%
  - Test Data Accuracy: 84%

- Based on the accuracy scores, the DecisionTreeClassifier achieves the highest accuracy on the training data, but it has the lowest accuracy on the test data, suggesting potential overfitting.
- The PCA with DecisionTreeClassifier also shows signs of overfitting, as it performs perfectly on the training data but significantly worse on the test data.
- Among the remaining models, both the GridSearchCV model with DecisionTreeClassifier and the LogisticRegression model have similar accuracy scores on both the training and test data. These two models generalize well and have a balance between precision and recall for both classes.

#### which model ?

- Given that the primary business goal is likely to have a model that generalizes well to new, unseen data, the production model choice should be based on the balance between performance on the test data and avoiding overfitting. In this case, the LogisticRegression model would be a suitable choice for production.